In [1]:
import pandas as pd
import datetime

In [2]:
df_train = pd.read_csv("./data/predict-energy-behavior-of-prosumers/train.csv")
df_client=pd.read_csv("./data/predict-energy-behavior-of-prosumers/client.csv")
df_electricity=pd.read_csv("./data/predict-energy-behavior-of-prosumers/electricity_prices.csv")



In [10]:
df_train.sample(10000)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
486753,11,0,2,9.513,1,2022-02-08 13:00:00,160,486753,44
1532366,0,1,1,0.000,0,2022-12-30 09:00:00,485,1532366,4
640201,13,0,1,8.689,1,2022-03-28 18:00:00,208,640201,50
284427,3,0,1,54.098,1,2021-12-05 00:00:00,95,284427,11
1302519,14,0,1,38.488,1,2022-10-20 22:00:00,414,1302519,53
...,...,...,...,...,...,...,...,...,...
541155,11,1,3,5216.150,1,2022-02-25 15:00:00,177,541155,48
547591,13,0,1,6.481,1,2022-02-27 15:00:00,179,547591,50
1468551,13,0,3,153.727,1,2022-12-10 12:00:00,465,1468551,51
644128,1,0,3,0.000,0,2022-03-30 00:00:00,210,644128,7


In [4]:
df_client

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2
...,...,...,...,...,...,...,...
41914,1,15,51,415.60,0,2023-05-29,637
41915,3,15,161,2035.75,0,2023-05-29,637
41916,0,15,15,620.00,1,2023-05-29,637
41917,1,15,20,624.50,1,2023-05-29,637


In [5]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1
...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637


In [6]:
df_electricity["origin_date"].nunique()

15286

In [7]:
df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_electricity["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

In [8]:
df_client["date_str"] = df_client["date"].apply(lambda x : str(x))
df_electricity["date_str"] = df_electricity["date"].apply(lambda x : str(x))

## Feature Engineering

In [9]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


## Creation colonne

In [19]:
df_train["price_elec"] = -1

for unique_date in df_train["datetime"].unique()[:10]:
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train_filtered["price_elec"] = elec_price

1
TOTO
1
TOTO
1
TOTO
1
TOTO
1
TOTO
1
TOTO
1
TOTO
1
TOTO
1
TOTO
1
TOTO


In [17]:
df_train.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec
count,2.018352e+06,2.018352e+06,2.018352e+06,2.017824e+06,2018352.0,2.018352e+06,2.018352e+06,2.018352e+06,2018352.0
mean,7.297034e+00,5.368261e-01,1.898927e+00,2.748556e+02,0.5,3.218746e+02,1.009176e+06,3.304538e+01,-1.0
std,4.780990e+00,4.986421e-01,1.081766e+00,9.095024e+02,0.5,1.826343e+02,5.826482e+05,1.959059e+01,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-1.0
25%,3.000000e+00,0.000000e+00,1.000000e+00,3.780000e-01,0.0,1.660000e+02,5.045878e+05,1.600000e+01,-1.0
50%,7.000000e+00,1.000000e+00,2.000000e+00,3.113300e+01,0.5,3.230000e+02,1.009176e+06,3.300000e+01,-1.0
75%,1.100000e+01,1.000000e+00,3.000000e+00,1.802062e+02,1.0,4.790000e+02,1.513763e+06,5.000000e+01,-1.0
max,1.500000e+01,1.000000e+00,3.000000e+00,1.548027e+04,1.0,6.370000e+02,2.018351e+06,6.800000e+01,-1.0


In [61]:
def find_time(group):
  df_electricity[df_electricity["origin_date"] == group.iloc[0]["datetime"]].iloc[0]["euros_per_mwh"]
  

df_train.groupby("datetime")["electricity price"] = df_train.groupby("datetime").apply(find_time)

IndexError: single positional indexer is out-of-bounds

In [49]:
import time

def find_electricity_price(line):
  if int(line.name) % 2000 == 0 : 
    print(line.name, "/", df_train.shape[0],
          (line.name/df_train.shape[0])*100, "%", end="\r")
    time.sleep(2)
  return df_electricity[df_electricity["origin_date"] == line["datetime"]].iloc[0]["euros_per_mwh"]

df_train["euros_per_mwh"] = df_train.apply(find_electricity_price, axis = 1)
df_train

KeyboardInterrupt: 

In [19]:
# df_train["euros_per_mwh"] = 
df_train.apply(lambda x : df_electricity[df_electricity["origin_date"] == x["datetime"]].values, axis = 1)

KeyboardInterrupt: 